In [ ]:
import tkinter as tk
from tkinter import messagebox, ttk
import heapq
from tkcalendar import DateEntry
from PIL import Image, ImageTk
from datetime import datetime
import json
import os
import re

# Define Task class
class Task:
    def __init__(self, name, deadline, priority):
        self.name = name
        self.deadline = deadline
        self.priority = priority
        self.days_left = self.calculate_days_left()

    def __lt__(self, other):
        return self.priority < other.priority

    def __repr__(self):
        return f"Task(name={self.name}, deadline={self.deadline}, priority={self.priority}, days_left={self.days_left})"
    
    def calculate_days_left(self):
        try:
            # Convert the deadline string to a datetime object
            deadline_date = datetime.strptime(self.deadline, "%m/%d/%y").date()
            today = datetime.today().date()
            if deadline_date < today:
                messagebox.showerror("Invalid Deadline", "The deadline cannot be a past date.")
                raise ValueError("Deadline cannot be in the past.")
            delta = deadline_date - today
            return delta.days
        except ValueError:
            return None


# AVL Tree implementation (same as before)
class AVLTreeNode:
    def __init__(self, task):
        self.task = task
        self.left = None
        self.right = None
        self.height = 1


class AVLTree:
    def __init__(self):
        self.root = None

    def insert(self, root, task):
        if not root:
            return AVLTreeNode(task)

        if task.name < root.task.name:
            root.left = self.insert(root.left, task)
        else:
            root.right = self.insert(root.right, task)

        root.height = 1 + max(self.get_height(root.left), self.get_height(root.right))

        balance = self.get_balance(root)

        # Left Heavy
        if balance > 1 and task.name < root.left.task.name:
            return self.right_rotate(root)

        # Right Heavy
        if balance < -1 and task.name > root.right.task.name:
            return self.left_rotate(root)

        # Left-Right case
        if balance > 1 and task.name > root.left.task.name:
            root.left = self.left_rotate(root.left)
            return self.right_rotate(root)

        # Right-Left case
        if balance < -1 and task.name < root.right.task.name:
            root.right = self.right_rotate(root.right)
            return self.left_rotate(root)

        return root

    def left_rotate(self, z):
        y = z.right
        T2 = y.left

        y.left = z
        z.right = T2

        z.height = 1 + max(self.get_height(z.left), self.get_height(z.right))
        y.height = 1 + max(self.get_height(y.left), self.get_height(y.right))

        return y

    def right_rotate(self, z):
        y = z.left
        T3 = y.right

        y.right = z
        z.left = T3

        z.height = 1 + max(self.get_height(z.left), self.get_height(z.right))
        y.height = 1 + max(self.get_height(y.left), self.get_height(y.right))

        return y

    def get_height(self, root):
        if not root:
            return 0
        return root.height

    def get_balance(self, root):
        if not root:
            return 0
        return self.get_height(root.left) - self.get_height(root.right)

    def search(self, root, name):
        if root is None or root.task.name == name:
            return root

        if name < root.task.name:
            return self.search(root.left, name)
        return self.search(root.right, name)

    def inorder(self, root):
        if root:
            self.inorder(root.left)
            print(root.task)
            self.inorder(root.right)

    def remove(self, root, name):
        if root is None:
            return root

        if name < root.task.name:
            root.left = self.remove(root.left, name)
        elif name > root.task.name:
            root.right = self.remove(root.right, name)
        else:
            if not root.left:
                return root.right
            elif not root.right:
                return root.left

            temp = self.get_min_value_node(root.right)
            root.task = temp.task
            root.right = self.remove(root.right, temp.task.name)

        root.height = 1 + max(self.get_height(root.left), self.get_height(root.right))

        balance = self.get_balance(root)

        if balance > 1 and self.get_balance(root.left) >= 0:
            return self.right_rotate(root)

        if balance < -1 and self.get_balance(root.right) <= 0:
            return self.left_rotate(root)

        if balance > 1 and self.get_balance(root.left) < 0:
            root.left = self.left_rotate(root.left)
            return self.right_rotate(root)

        if balance < -1 and self.get_balance(root.right) > 0:
            root.right = self.right_rotate(root.right)
            return self.left_rotate(root)

        return root

    def get_min_value_node(self, root):
        if root is None or root.left is None:
            return root
        return self.get_min_value_node(root.left)


# Priority Queue (using heapq)
class PriorityQueue:
    def __init__(self):
        self.heap = []

    def push(self, task):
        heapq.heappush(self.heap, task)

    def pop(self):
        if self.heap:
            return heapq.heappop(self.heap)
        return None

    def peek(self):
        if self.heap:
            return self.heap[0]
        return None

class TaskSchedulerApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Task Scheduler")
        self.root.geometry("1400x750")

        self.welcome_image=None
        
        self.background_photo=None # Keep reference here
        
        self.avl_tree = AVLTree()
        self.priority_queue = PriorityQueue()

        # Show the welcome page
        self.create_welcome_page()


    def create_welcome_page(self):
        try:
            #image_path = r"C:\Users\khizz\front image.jpg"
            if not os.path.exists("welcome.jpg"):
                raise FileNotFoundError(f"Image file not found at {"welcome.jpg"}")
            welcome_image = Image.open("welcome.jpg")
            welcome_image = welcome_image.resize((1400, 750), Image.Resampling.LANCZOS)
            self.welcome_photo = ImageTk.PhotoImage(welcome_image) 
        except Exception as e:
            print(f"Error loading image: {e}")
            self.welcome_photo = None


        # Create a canvas for the image
        self.canvas = tk.Canvas(self.root, width=1400, height=750)
        self.canvas.pack()

        if self.welcome_photo:
            self.canvas.create_image(0, 0, anchor=tk.NW, image=self.welcome_photo)
        else:
            self.canvas.config(bg="gray")  # Fallback background color

        start_button = tk.Button(self.root, text="Start", font=("Helvetica", 18, "bold italic underline"),
                         command=self.show_main_window, bg="black", fg="white", width=10, height=1,
                         bd=5, relief="solid", highlightbackground="blue")

        
        self.canvas.create_window(700, 625, window=start_button)  # (x, y) coordinates at the top center of the canvas

    def show_main_window(self):
        self.canvas.destroy()
        self.root.title("Task Scheduler")
        self.root.geometry("1400x750")

        #bg_image_path = r"C:\Users\khizz\bg.jpg"
        try:
            bg_image = Image.open("bg.jpg")
            bg_image = bg_image.resize((1400, 750), Image.Resampling.LANCZOS)
            self.background_photo = ImageTk.PhotoImage(bg_image)
        except Exception as e:
            print(f"Error loading background image: {e}")

        # Create a canvas and add the background image
        self.canvas = tk.Canvas(self.root, width=1400, height=750)
        self.canvas.pack(fill="both", expand=True)
        if self.background_photo:
            self.canvas.create_image(0, 0, anchor=tk.NW, image=self.background_photo)

        self.create_main_window()
        self.load_tasks()


    def create_main_window(self): 
        self.welcome_label = tk.Label(
            self.root,
            text="Welcome to Task Scheduler",
            font=("Cooper Black", 28, "bold italic underline"),
            bg="black",  # Background color for the label
            fg="#87CEEB"
        )
        self.canvas.create_window(700, 50, window=self.welcome_label)

        # Task Name
        self.task_name_label = tk.Label(self.root, text="Task Name:", font=("Cooper Black", 18, "bold italic"), fg="#87CEEB", bg="black")
        self.canvas.create_window(300, 150, window=self.task_name_label)

        self.task_name_entry = tk.Entry(self.root, width=50)
        self.canvas.create_window(700, 150, window=self.task_name_entry)

        # Deadline
        self.deadline_label = tk.Label(self.root, text="Deadline:", font=("Cooper Black", 18, "bold italic"), fg="#87CEEB", bg="black")
        self.canvas.create_window(300, 200, window=self.deadline_label)

        self.deadline_entry = DateEntry(self.root, width=50)
        self.canvas.create_window(700, 200, window=self.deadline_entry)

        # Priority
        self.priority_label = tk.Label(self.root, text="Priority:", font=("Cooper Black", 18, "bold italic"), fg="#87CEEB", bg="black")
        self.canvas.create_window(300, 250, window=self.priority_label)

        self.priority_entry = tk.Entry(self.root, width=50)
        self.canvas.create_window(700, 250, window=self.priority_entry)

        # Buttons
        button_style = {"width": 10,"height" :1, "bg": "black", "fg": "#87CEEB", "font": ("Cooper Black", 16, "bold italic")}
        self.add_button = tk.Button(self.root, text="Add Task", command=self.add_task, **button_style)
        self.canvas.create_window(500, 300, window=self.add_button)

        self.update_button = tk.Button(self.root, text="Update Task", command=self.update_task, **button_style)
        self.canvas.create_window(700, 300, window=self.update_button)

        self.delete_button = tk.Button(self.root, text="Delete Task", command=self.delete_task, **button_style)
        self.canvas.create_window(500, 350, window=self.delete_button)

        self.search_button = tk.Button(self.root, text="Search Task", command=self.search_task, **button_style)
        self.canvas.create_window(700, 350, window=self.search_button)

        self.sort_button = tk.Button(self.root, text="Sort Tasks", command=self.sort_tasks, **button_style)
        self.canvas.create_window(500, 400, window=self.sort_button)

        self.clear_button = tk.Button(self.root, text="Clear Input", command=self.clear_input, **button_style)
        self.canvas.create_window(700, 400, window=self.clear_button)

        # Treeview
        self.task_table = ttk.Treeview(self.root, columns=("Task Name", "Deadline", "Priority", "Days Remaining"), show="headings")
        self.task_table.heading("Task Name", text="Task Name")
        self.task_table.heading("Deadline", text="Deadline")
        self.task_table.heading("Priority", text="Priority")
        self.task_table.heading("Days Remaining", text="Days Remaining")

        style = ttk.Style()
        style.configure("Treeview", background="gray", fieldbackground="black", foreground="white",font=("Arial",12))
        style.configure("Treeview.odd", background="lightgray")
        style.configure("Treeview.even", background="white")

        style.configure("Treeview.Heading", 
                background="black",  # Heading background color
                foreground="black",     # Heading text color
                font=("Helvetica", 14, "bold"))  # Hea


        self.canvas.create_window(700, 550, window=self.task_table, width=1300, height=200)
        #self.task_table.pack(pady=50)
        self.task_table.bind("<ButtonRelease-1>", self.select_task_for_update_or_delete)


    def add_task(self):
        task_name = self.task_name_entry.get().strip().lower()

        if not self.is_valid_task_name(task_name):
            messagebox.showerror("Invalid Input", "Task name is invalid. It must be at least 3 characters long and not contain special characters.")
            return
            
        deadline = self.deadline_entry.get()
        try:
            deadline_date = datetime.strptime(deadline, "%m/%d/%y").date()
            today_date = datetime.today().date()
            if deadline_date < today_date:
                messagebox.showerror("Invalid Deadline", "The deadline cannot be a past date.")
                return
        except ValueError:
            messagebox.showerror("Invalid Input", "Deadline is not in the correct format.")
            return
            
        priority = None
        try:
            priority = int(self.priority_entry.get())
        except ValueError:
            messagebox.showerror("Invalid input","priority must be a valid integer.")

        if not deadline.strip() or priority is None:
            messagebox.showerror("Input Error", "Please fill in all fields.")
            return

        task = Task(task_name, deadline, priority)

        self.avl_tree.root = self.avl_tree.insert(self.avl_tree.root, task)
        self.priority_queue.push(task)
        
        self.refresh_task_table()
        self.clear_input()

        # Save tasks to file
        self.save_tasks()

        messagebox.showinfo("Success", "Task added successfully.")

    def update_task(self):
        task_name = self.task_name_entry.get()
        if not self.is_valid_task_name(task_name):
            messagebox.showerror("Invalid Input", "Task name is invalid. It must be at least 3 characters long and not contain special characters.")
            return
        node = self.avl_tree.search(self.avl_tree.root, task_name)

        if node:
            task=node.task

            new_deadline = self.deadline_entry.get()
            try:
                deadline_date = datetime.strptime(new_deadline, "%m/%d/%y").date()
                today_date = datetime.today().date()

            # Validate that the new deadline is not in the past
                if deadline_date < today_date:
                    messagebox.showerror("Invalid Deadline", "The new deadline cannot be a past date.")
                    return
                task.deadline = new_deadline  # Update deadline if valid
            except ValueError:
                messagebox.showerror("Invalid Input", "Deadline is not in the correct format.")
                return
                
            new_priority = self.priority_entry.get()

            if new_priority.strip():  # Only update priority if it's provided

                try:
                    task.priority = int(new_priority)
                except ValueError:
                    messagebox.showinfo("Invalid input, priority must be a valid integer.")
                
            task.days_left = task.calculate_days_left() 
            
            self.avl_tree.root = self.avl_tree.remove(self.avl_tree.root, task_name)
            self.priority_queue.heap = [t for t in self.priority_queue.heap if t.name != task_name]  # Remove from Priority Queue
            heapq.heapify(self.priority_queue.heap)
            
            self.avl_tree.root = self.avl_tree.insert(self.avl_tree.root, task)
            self.priority_queue.push(task)


            # Refresh the table to reflect the changes
            self.refresh_task_table()
            self.clear_input()

            # Save tasks to file
            self.save_tasks()

            messagebox.showinfo("Success", "Task updated successfully.")
        else:
            messagebox.showerror("Task Not Found", "No task found with that name.")

    def is_valid_task_name(self, task_name):
        if len(task_name.strip()) < 3:
            return False
        if re.search(r'[^a-zA-Z0-9\s]', task_name):  # Check for special characters
            return False
        return True

    def delete_task(self):
        task_name = self.task_name_entry.get()
        node = self.avl_tree.search(self.avl_tree.root, task_name)

        if node:
            self.avl_tree.root = self.avl_tree.remove(self.avl_tree.root, task_name)

            self.priority_queue.heap = [task for task in self.priority_queue.heap if task.name != task_name]
            heapq.heapify(self.priority_queue.heap)

            # Refresh the table to reflect the changes
            self.refresh_task_table()
            self.clear_input()

            # Save tasks to file
            self.save_tasks()

            messagebox.showinfo("Success", "Task deleted successfully.")
        else:
            messagebox.showerror("Task Not Found", "No task found with that name.")
            
    def search_task(self):
        task_name = self.task_name_entry.get().strip().lower()
        node = self.avl_tree.search(self.avl_tree.root, task_name)

        if node:
            task = node.task
            messagebox.showinfo(
                "Task Found", 
                f"Task Found!\n\nTask Name: {task.name}\nDeadline: {task.deadline}\nPriority: {task.priority}\nDays Remaining: {task.days_left}"
            )
        else:
            messagebox.showerror("Task Not Found", f"No task found with the name: {self.task_name_entry.get()}")

    def sort_tasks(self):
        sorted_tasks = []
        while self.priority_queue.heap:
            sorted_tasks.append(heapq.heappop(self.priority_queue.heap))

        for row in self.task_table.get_children():
            self.task_table.delete(row)

        for task in sorted_tasks:
            self.task_table.insert("", "end", values=(task.name, task.deadline, task.priority, task.days_left))

        for task in sorted_tasks:
            self.priority_queue.push(task)

        messagebox.showinfo("Success", "Tasks sorted by priority.")

    def clear_input(self):
        self.task_name_entry.delete(0, tk.END)
        self.deadline_entry.delete(0, tk.END)
        self.priority_entry.delete(0, tk.END)
        self.task_name_entry.focus()
        self.deadline_entry.focus()   # You can uncomment this if you want to focus on the deadline field after clearing


    def refresh_task_table(self):
        for row in self.task_table.get_children():
            self.task_table.delete(row)

        # Add tasks to the table from AVL tree
        self._populate_task_table(self.avl_tree.root)
        
    def select_task_for_update_or_delete(self, event):
        selected_item = self.task_table.selection()

        if not selected_item:  
            return
        task_name = self.task_table.item(selected_item[0], 'values')[0]
        node = self.avl_tree.search(self.avl_tree.root, task_name)

        if node:
            task = node.task
            self.task_name_entry.delete(0, tk.END)
            self.task_name_entry.insert(0, task.name)

            self.deadline_entry.delete(0, tk.END)
            self.deadline_entry.insert(0, task.deadline)

            self.priority_entry.delete(0, tk.END)
            self.priority_entry.insert(0, str(task.priority))

    def _populate_task_table(self, root):
        if root:
            self._populate_task_table(root.left)
            self.task_table.insert("", "end", values=(root.task.name, root.task.deadline, root.task.priority, root.task.days_left))
            self._populate_task_table(root.right)

    def save_tasks(self):
        tasks = []
        self._collect_tasks(self.avl_tree.root, tasks)
        tasks_data = [{
            'name': task.name,
            'deadline': task.deadline,
            'priority': task.priority,
        } for task in tasks]

        # Save the tasks to a file
        with open("tasks.json", "w") as file:
            json.dump(tasks_data, file)

    def _collect_tasks(self, root, tasks):
        if root:
            self._collect_tasks(root.left, tasks)
            tasks.append(root.task)
            self._collect_tasks(root.right, tasks)

    def load_tasks(self):
        if os.path.exists("tasks.json"):
            with open("tasks.json", "r") as file:
                try:
                    tasks_data = json.load(file)
                    for task_data in tasks_data:

                        name = task_data.get('name')
                        deadline = task_data.get('deadline')
                        priority = task_data.get('priority')
                        
                        if name and deadline and priority:
                            task = Task(name, deadline, priority)
                            self.avl_tree.root = self.avl_tree.insert(self.avl_tree.root, task)
                            self.priority_queue.push(task)  # Add task to priority queue too

                except json.JSONDecodeError:
                    messagebox.showerror("Error", "Unable to load tasks. The file may be missing or corrupted.")

            self.refresh_task_table()

def main():
    root = tk.Tk()
    app = TaskSchedulerApp(root)
    root.mainloop()

main()
